# Lecture 23 - Twitter Bot
In this notebook we will learn how to build a Twitter bot using the Twitter API. 

<ol type = 1>
<li> Basic Functions</li>
    <ol type = a>
        <li> Tweeting</li>
        <li> Retweeting</li>
        <li> Liking</li>
        <li> Replying</li>
    </ol>
<li> Tweeting Fake Text</li>
    <ol type = a>
        <li> Huggingface</li>
        <li> GPT-3</li>
    </ol>
<li> Perpetual Tweeting</li>  
<li> Monday Motivation Bot </li>
    
<li> Pacing and Leading Bot</li>

<li> Bot-Bot Conversation</li>


</ol>

    
    
This notebook will not work in Colab.  Please run it on your local machine.

# Clones, installs, and imports


## Clone GitHub Repository

You will have to clone the repository to your local machine before running this notebook.  This can be done from a terminal by typing `git clone https://github.com/zlisto/social_media_analytics.git`.

If you have already cloned this repository before, but want to update to the current version, from a terminal change directory to `social_media_analytics` and then type `git pull`.

## Install Requirements 

In [ ]:
!pip install -r requirements.txt


## API Keys

After cloning or pulling the repository, they file `scripts/config_bot.py` will be overwritten to a blank file.  You will need to edit the file by pasting in your Twitter API keys, Twitter user id, and OpenAI API key as follows: 

1. `api_key = 'your api key here as a string'`.  

2. `api_secret = 'your api secret here as a string'`. 

3. `access_key = 'your access key here as a string'`. 

4. `access_secret = 'your api key here as a string'`. 

5. `user_id = 'your Twitter id here as a string'`

6. `OPENAI_API_KEY = 'your OpenAI API key`

You can access your Twitter API keys here: https://developer.twitter.com/en/portal/dashboard, go to your project, and select Keys and Tokens. 


You can find your Twitter user id by going to this site and pasting your screen name: https://tweeterid.com/.


We can use GPT-3 to generate text. First you need to create an account with OpenAI here: https://auth0.openai.com/u/signup?state=hKFo2SBWS3JUVEdmQmdzZXo5ckhpY3R5NEFlc2NPWWc3WHhvRqFur3VuaXZlcnNhbC1sb2dpbqN0aWTZIG9kTDB4LV83aEdnN3pRU3VUYnVZemlnZkFURFo2RDhno2NpZNkgRFJpdnNubTJNdTQyVDNLT3BxZHR3QjNOWXZpSFl6d0Q

Once you have an account, copy your API key from here: https://beta.openai.com/account/api-keys



## Import Packages

The important import is from `scripts.bot` where we have the bot command methods.

In [1]:
%load_ext autoreload
%autoreload 2

from scripts.api import *
from scripts.bot import *
from transformers import pipeline
import openai
import scripts.TextAnalysis as ta
import textwrap as tr

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import datetime
import requests
import json

pd.set_option("display.max_colwidth", None)


# Get OAuth Object

First we need to give the code access to control your Twitter acccount.  Run the code, then open the link provided.  Copy the pin in the webpage that opens and paste it here, then press Shift+Enter to execute the code (for some reason pressing the Run button makes it freeze).


In [6]:
oauth = Bot.fetch_auth()


Please paste the pin from 
 https://api.twitter.com/oauth/authorize?oauth_token=GmQCggAAAAAAGzW4AAABgG7l1Cs 
 here: 6267163


# Basic Functions

The code below allows you to perform basic Twitter functions, like tweeting, retweeting, liking, and replying.



## Tweeting

Set the `text` equal to the text you want to tweet.  Then use the `Bot.tweet` method to post the tweet.

In [ ]:
text = "i love cheeseballs.  The ones in the cylinder box by Planters.  #Cheeseballs"

Bot.tweet(text, oauth=oauth)

## Retweeting

Set `tweet_id` to the id of the tweet you want to retweet.  Then use the `retweet` method to retweet it.

In [ ]:
df = History.fetch_recent("cheeseballs")
df.head(n=4)


### Get tweet id and retweet it
Select the row of the tweet you like, then get its `tweet_id` and retweet it with the `Bot.retweet` method.

In [ ]:
tweet_row= 0
tweet_id = df.iloc[tweet_row].id

Bot.retweet(tweet_id, oauth=oauth)

## Liking

Select the row of the tweet you like, then get its `tweet_id`.  Then use the `Bot.like` method to like it.

In [ ]:
tweet_row= 2
tweet_id = df.iloc[tweet_row].id
Bot.like(tweet_id, oauth=oauth)

## Replying

Select the row of the tweet you want to reply to and set `text` equal to the text you want to reply with.  Then use the `Bot.tweet` method with the `reply_to` parameter equal to `tweet_id` to reply to the tweet.

In [ ]:
tweet_row= 2
tweet_id = df.iloc[tweet_row].id
text = "Nice :)"

Bot.tweet(text=text, 
          reply_to=tweet_id, 
          oauth=oauth)

# Fake Tweets

We can use transformer language models to generate fake tweets, and then tweet them.

## Load language model 

Choose the language model you want to use.  There are two options here:
    1. Huggingface
    2. GPT-3 (OpenAI API access required)

### Huggingface model

Choose your `model_name` from the hugging face website: https://huggingface.co/models.
Create the `generator` and then set all the sampling parameters.  Choose the `input_text` to start the tweet.

In [ ]:
model_name = "huggingtweets/zlisto"
generator = pipeline('text-generation',model=model_name)


In [ ]:
input_text = "I feel sick, but I dont think it's"
max_length = 1000
num_return_sequences = 5
temperature = 1
p = 0.9

In [ ]:
outputs =generator(input_text, 
                       max_length=max_length,
                       num_return_sequences= num_return_sequences,
                       temperature = float(temperature),
                       do_sample = True,
                       top_p = p,
                       pad_token_id=50256)

print(f"Fake Tweets"+50*"-")
ta.display_text(outputs)

Choose the tweet you want to tweet and tweet it!

In [ ]:
tweet_index = 0
text = outputs[tweet_index]['generated_text']

Bot.tweet(text, oauth=oauth)

### GPT-3 Model



Set all the sampling parameters.  Choose the `input_text` to start the tweet.  You can write something like `"Write a positive tweet about Yale."`

In [2]:
input_text = "Write a positive tweet about Yale"
max_length = 1000
temperature = 1
p = 0.9
engine =  "text-davinci-002"

In [4]:
responses = openai.Completion.create(
  engine=engine,
  prompt= input_text,
  temperature=temperature,
  max_tokens=max_length,
  top_p=p,
  frequency_penalty=0,
  presence_penalty=0,
)

#print text
text = responses['choices'][0]['text']
print(f"GPT-3 Text:\n{tr.fill(text,width = 50)}")

GPT-3 Text:
    I'm so proud to be a part of the Yale
community! It's a truly special place filled with
amazing people.


Tweet text!

In [7]:
Bot.tweet(text,oauth = oauth)

# Perpetual Tweeting

We use a `while` loop combined with the `sleep` function to make the bot tweet perpetually.  The bot will tweet, then sleep for a random amount of time, continuously in a loop.  We can use a language model to create the tweets.

The mean sleep time is `tsleep_mean`, measured in seconds. We then add some noise to it to make it look more random to obtain the sleep time `tsleep`.  We also set `tweet_max` equal to the maximum number of tweets to generate.

## Perpetual tweeting with huggingface

In [9]:
model_name = "huggingtweets/zlisto"
generator = pipeline('text-generation',model=model_name)

max_length = 1000
num_return_sequences = 1
temperature = 1
p = 0.9
engine =  "text-davinci-002"

tsleep_mean = 5  #mean sleep time in seconds
tweet_max = 2
input_text = ""

In [10]:
c = 0
while True:
    c+=1
    if c>tweet_max:break  #stop after tweet_max tweets
    outputs =generator(input_text, 
                       max_length=max_length,
                       num_return_sequences= num_return_sequences,
                       temperature = float(temperature),
                       do_sample = True,
                       top_p = p,
                       pad_token_id=50256)
    text = outputs[0]['generated_text']
    print(f"{c}: {text}")
    Bot.tweet(text,oauth = oauth)  
    
    tsleep = tsleep_mean + np.random.uniform(low=0.0, high=3)
    print(f"Sleeping for {tsleep:.2f} seconds\n")
    time.sleep(tsleep)

1: I had this issue with the logo. Maybe I should get one replaced #Corona
Sleeping for 5.55 seconds

2: Tbh my name is Eddy Kornacki and I'm trying to win the heavyweight belt.
Sleeping for 6.79 seconds



## Perpetual tweeting with GPT-3

In [16]:
max_length = 1000
temperature = 1
p = 0.9

tsleep_mean = 5  #mean sleep time in seconds
tweet_max = 2
input_text = "Write a funny tweet about Yale being better than Harvard."

In [17]:
c = 0
while True:
    c+=1
    if c>tweet_max:break  #stop after tweet_max tweets
    responses = openai.Completion.create(
      engine=engine,
      prompt= input_text,
      temperature=temperature,
      max_tokens=max_length,
      top_p=p,
      frequency_penalty=0,
      presence_penalty=0,
    )

    #print text
    text = responses['choices'][0]['text']
    text = text.strip()  #remove leading and trailing white spaces

    print(f"{c}: {text}")
    Bot.tweet(text,oauth = oauth)    
    
    tsleep = tsleep_mean + np.random.uniform(low=0.0, high=3)
    print(f"Sleeping for {tsleep:.2f} seconds\n")
    time.sleep(tsleep)

1: Yale is better than Harvard because we have a superior sense of humor!
Sleeping for 7.74 seconds

2: Yale is better than Harvard because our students are smarter, our campus is prettier, and our endowment is bigger. #GoBulldogs
Sleeping for 5.60 seconds



# Monday Motivation Bot

This bot will tweet something about #MondayMotivation each Monday.

In [21]:
max_length = 1000
temperature = 1
p = 0.9

tsleep_mean = 5  #mean sleep time in seconds
tweet_max = 2
input_text = "Write a clever tweet using the hashtag #MondayMotivation."

In [22]:
c = 0
while True:
    c+=1
    if c>tweet_max: break  #stop after tweet_max tweets
    responses = openai.Completion.create(
      engine=engine,
      prompt= input_text,
      temperature=temperature,
      max_tokens=max_length,
      top_p=p,
      frequency_penalty=0,
      presence_penalty=0,
    )

    #print text
    text = responses['choices'][0]['text']
    text = text.strip()
    
    print(f"{c}: {text}")
    Bot.tweet(text,oauth = oauth)   
    
    tsleep = tsleep_mean + np.random.uniform(low=0.0, high=2)
    print(f"Sleeping for {tsleep:.2f} seconds\n")
    time.sleep(tsleep)

1: No one ever said life was easy. But it's a lot easier when you have positive #MondayMotivation.
Sleeping for 5.70 seconds

2: I'm #MondayMotivated to get things done and make this week a great one!
Sleeping for 6.54 seconds



# Pacing and Leading Bot

This bot will implement a pacing and leading policy.  Set `sentiments` equal to the sentiment for each tweet as a list and set `topic` equal to the topic of your perusasion.  The sentiment should be a number between 0 and 5.

In [24]:
max_length = 1000
temperature = 1
p = 0.9

tsleep_mean = 5  #mean sleep time in seconds
topic = "Kanye West"
sentiments = list(range(0,6))

print(f"sentiments = {sentiments}")


sentiments = [0, 1, 2, 3, 4, 5]


In [26]:
print(f"Pacing and leading about {topic}")
c = 0
for sentiment in sentiments:
    c+=1
    input_text = f"Write a tweet with sentiment {sentiment}/5.0 about {topic}."

    responses = openai.Completion.create(
      engine=engine,
      prompt= input_text,
      temperature=temperature,
      max_tokens=max_length,
      top_p=p,
      frequency_penalty=0,
      presence_penalty=0,
    )

    #print text
    text = responses['choices'][0]['text']
    text = text.strip()
    
    print(f"Tweet {c}: sentiment = {sentiment}/5.0: {text}")
    Bot.tweet(text,oauth = oauth)   
    
    tsleep = tsleep_mean + np.random.uniform(low=0.0, high=2)
    print(f"Sleeping for {tsleep:.2f} seconds\n")
    time.sleep(tsleep)

Pacing and leading about Kanye West
Tweet 1: sentiment = 0/5.0: Kanye West is the worst.
Sleeping for 5.29 seconds

Tweet 2: sentiment = 1/5.0: Kanye West is a talentless egomaniac who is only famous because he's dating Kim Kardashian.
Sleeping for 5.68 seconds

Tweet 3: sentiment = 2/5.0: I can't stand Kanye West. He's so arrogant and always talking about himself.
Sleeping for 5.27 seconds

Tweet 4: sentiment = 3/5.0: I'm not the biggest fan of Kanye, but I have to admit he's a genius when it comes to music. #Respect
Sleeping for 5.62 seconds

Tweet 5: sentiment = 4/5.0: I'm not a huge fan of Kanye, but I have to admit he's a genius. #Respect
Sleeping for 5.33 seconds

Tweet 6: sentiment = 5/5.0: Kanye West is one of the most influential artists of our generation. His music has changed the landscape of hip hop and popular culture.
Sleeping for 6.38 seconds



# Bot Conversation

We can make GPT-3 talk to itself.  Name `speaker0` and `speaker1` and describe them in  `desc0` and `desc1`.  Then set the first messages between them as `text0` and `text1`.  Set how many messages you want to generate with `msg_max`.  Then run the code and see what the spekaers have to say to each other.

In [74]:
msg_max = 6
speaker0 = "Goku"
speaker1 = "Vegeta"

context = f"The following is a conversation between {speaker0} and {speaker1}."

desc0 = f"{speaker0} is from Dragonball Z and is kind and funny."
desc1 = f"{speaker1} is from Dragonball Z and is angry and jealous."

text0 = f"{speaker0}: Hey Vegeta!"
text1 = f"{speaker1}: Go away Goku."

In [75]:
input_text = f"{context}\n{desc0}\n{desc1}\n\n{text0}\n{text1}"

print(input_text)

for i in range(msg_max):
    if i%2==0:
        speaker = speaker0
    elif i%2==1:
        speaker = speaker1

    input_text+=f"\n{speaker}:"
    responses = openai.Completion.create(
      engine=engine,
      prompt= input_text,
      temperature=temperature,
      max_tokens=max_length,
      top_p=p,
      frequency_penalty=0,
      presence_penalty=0.4,
    )   
    text = responses['choices'][0]['text']
    text = text.strip()
    #text +="\n"
    input_text += text
    print(f"{speaker}: {text}")
    time.sleep(2)


    
    


The following is a conversation between Goku and Vegeta.
Goku is from Dragonball Z and is kind and funny.
Vegeta is from Dragonball Z and is angry and jealous.

Goku: Hey Vegeta!
Vegeta: Go away Goku.
Goku: What's wrong?
Vegeta: I'm angry and jealous.
Vegeta: Goku: Hey Vegeta!
Vegeta: Go away Goku.
Goku:What's wrong?
Vegeta: I'm angry and jealous.
Goku: Of what?
Vegeta: Of you! You're always the one who saves the day. I'm always the one who comes in second place.
Goku: That's not true! You're a great fighter!
Vegeta: But I'm not as strong as you.
Goku: Maybe not, but you're still pretty strong.
Vegeta: I guess.
Goku: Come on, let's go train together. I'll help you get stronger.
Vegeta: Alright.
Goku: See, that wasn't so bad.
Vegeta: Yeah, I guess not.
Vegeta: Thanks, Goku.
